In [26]:
import re
import json
import os
from tqdm.notebook import tqdm

In [2]:
def process(line):
    try:
        score, triplet = line.split(":", 1)
        score = float(score.strip())
        triplet = triplet.strip()[1:-2].split(";")
        triplet = list(map(lambda x: x.strip(), triplet))
    except Exception as e:
        print(line)
        raise e
    return score, triplet

In [30]:
samples = {}
pbar = tqdm(total=117590)
with open("predictions.txt") as f:
    line = f.readline()
    while line:
        pbar.update(1)
        context = line
        idx = context.split(" ", 1)[0]
        results = []
        line = f.readline()
        while line != "\n":
            line = line.replace(idx, "")
            score, triplet = process(line)
            if score > 0.5:
                results.append(triplet)
            line = f.readline()
            pbar.update(1)
        samples[idx] = results
        line = f.readline()


In [4]:
orig_data = [json.loads(line) for line in open("/harddisk/user/keminglu/pretrained_data_processed/wikipedia_with_mention_wo_title_simplified_aug_eval/corpus_filtered_test_prompt_rephrased").readlines()]

In [5]:
orig_data = [sample for sample in orig_data if sample['aug_type'] == 'aug_default']

In [31]:
data_for_carb = []
for sample in orig_data:
    triplets = json.loads(sample['targets'])['triplets']
    true = []
    for each in triplets:
        for r in each['relations']:
            true.append([each['tail'], each['head'], r])
    pred = [[h, t, r] for h, r, t in samples[sample['id']]] if sample['id'] in samples else []
    data_for_carb.append((sample['inputs'], true, pred))

In [32]:
data_for_carb[10]

('"Money So Big" (stylized as "Monëy So Big") is a song by American rapper Yeat from his debut studio album "Up 2 Me" (2021). It was produced by Trgc and Nest. It gained traction through the video-sharing platform TikTok and has become one of Yeat\'s most popular songs.',
 [['Yeat', 'Up 2 Me', 'performer']],
 [['`` Money So Big', 'as `` Monëy So Bi', 'stylized'],
  ["`` Money So Big '' ( stylized as `` Monëy So Big '' )",
   "a song by American rapper Yeat from his debut studio album `` Up 2 Me '",
   'is'],
  ["`` Money So Big '' ( stylized as `` Monëy So Big '' )",
   "a song by American rapper Yeat from his debut studio album `` Up 2 Me '' ( 2021 ) one of Yeat 's most popular song",
   'is'],
  ["`` Money So Big '' ( stylized as `` Monëy So Big '' )",
   'a song by American rapper Yeat from his debut studio album `` Up',
   'is'],
  ["`` Money So Big '' ( stylized as `` Monëy So Big '' )",
   "a song by American rapper Yeat from his debut studio album `` Up 2 Me '' one of Yeat 's mo

In [34]:
carb_dir = "/data/home/keminglu/local_workspace/CaRB/files"
with open(os.path.join(carb_dir, f"openie6_gold.tsv"), "w") as fg:
    with open(os.path.join(carb_dir, f"openie6_pred.tsv"), "w") as fp:
        for sent, gold, pred in data_for_carb:
            sent = sent.replace("\t", "")
            for h, t, r in gold:
                h, t, r = h.replace("\t", ""), t.replace("\t", ""), r.replace("\t", "")
                r = r.replace("_", "")
                fg.write(f"{sent}\t{r}\t{h}\t{t}\n")
            for h, t, r in pred:
                h, t, r = h.replace("\t", ""), t.replace("\t", ""), r.replace("\t", "")
                r = r.replace("_", " ")
                fp.write(f"{sent}\t1\t{r}\t{h}\t{t}\n")